# Evaluate Performance of model using Backtest Pipeline

In [ ]:
import pandas as pd
from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass # TODO, import from openstef when availavle

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# define properties of training/prediction. We call this a 'prediction_job' 
pj=PredictionJobDataClass(id=287,
        model='xgb',
        quantiles=[0.10,0.30,0.50,0.70,0.90],
        horizon_minutes=48*60,
        resolution_minutes=15,
        lat = 1, #should become optional
        lon = 1, #should become optional
        train_components=False,
        name='TestPrediction',
        model_type_group=None, # Note, this should become optional
        hyper_params={}, # Note, this should become optional
        feature_names=None, # Note, this should become optional
        forecast_type="demand", # Note, this should become optional
                  )

modelspecs = ModelSpecificationDataClass(id=pj['id'])

# Load input data
input_data = pd.read_csv('data/get_model_input_pid_287.csv', index_col='index', parse_dates=True)


In [ ]:
# Perform the backtest
n_folds = 2
forecast, model, train_data, validation_data, test_data = train_model_and_forecast_back_test(
    pj,
    modelspecs = modelspecs,
    input_data = input_data,
    training_horizons=[0.25, 47.0],
    n_folds=n_folds,
 )
# If n_folds>1, model is a list of models. In that case, only use the first model
if n_folds>1:
    model=model[0]

# Evaluate results

In [ ]:
for horizon in set(forecast.horizon):
    fig = forecast.loc[forecast.horizon==0.25,['quantile_P10','quantile_P30',
                    'quantile_P50','quantile_P70','quantile_P90','realised','forecast']].plot(asFigure=True,
                                                                                   title=f"Horizon: {horizon}")
    fig.update_traces(
         line=dict(color="green", width=1), fill='tonexty', fillcolor='rgba(0, 255, 0, 0.1)',
         selector=lambda x: 'quantile' in x.name and x.name != 'quantile_P10')
    fig.update_traces(
         line=dict(color="green", width=1),
         selector=lambda x: 'quantile_P10' == x.name)
    fig.update_traces(
         line=dict(color="red", width=2),
         selector=lambda x: 'realised' in x.name)
    fig.update_traces(
         line=dict(color="blue", width=2),
         selector=lambda x: 'forecast' in x.name)
    fig.show()

In [ ]:
forecast['err']=forecast['realised']-forecast['forecast']
mea = forecast.pivot_table(index='horizon', values=['err'], aggfunc=lambda x: x.abs().mean())
mea.index=mea.index.astype(str)
mea.plot(kind='bar',
          layout=dict(title='MAE',
                      xaxis=dict(title='horizon'),
                      yaxis=dict(title='MAE [MW]')))

In [ ]:
plot_feature_importance(model.feature_importance_dataframe)